In [ ]:
import pandas as pd
from google.colab import drive
import uuid
from sqlalchemy.sql.expression import null

In [ ]:
drive.mount ( '/content/drive' ,  force_remount= True )    
historico_path = '/content/drive/MyDrive/Bases/reporte Horas Historico.csv'
cursos_path = '/content/drive/MyDrive/Bases/reporte cursos.csv'

Mounted at /content/drive


##Historico

In [ ]:
df_historico = pd.read_csv(historico_path,on_bad_lines='skip',delimiter=";")
df_historico.shape

<ipython-input-28-526bed6898f5>:1: DtypeWarning: Columns (19,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_historico = pd.read_csv(historico_path,on_bad_lines='skip',delimiter=";")


(151819, 36)

##Cursos

In [ ]:
df_cursos = pd.read_csv(cursos_path,on_bad_lines='skip',delimiter=";")
df_cursos.shape

(32191, 22)

#Division

##Metodos

In [ ]:
def getId(parent, name, value, id_col):
    res = parent[parent[name] == value]
    if len(res) == 0:
        return None
    else:
        return res[id_col].values[0]

def restructurar(df, clasesViejas,clasesAlteradas,clasesNuevas, clasesForaneas):
  remove = df.drop(clasesViejas, axis=1)
  newdf = df.drop(list(remove.columns), axis=1)
  newdf = newdf.drop_duplicates()

  for j in range(len(clasesNuevas)):
    res = []
    obj = clasesNuevas[j]
    for i in range(len(newdf)):
      if obj['value']:
        res.append(uuid.uuid4().hex[:14])
      else: res.append(obj['value'])
    newdf[obj['name']] = res

  for i in range(len(clasesViejas)):
    newdf[clasesAlteradas[i]] = newdf[clasesViejas[i]]
  newdf = newdf.drop(clasesViejas, axis=1)

  for i in range(len(clasesForaneas)):
    parent = clasesForaneas[i]['parent']
    name = clasesForaneas[i]['name']
    ref = clasesForaneas[i]['ref']
    newdf[name] = newdf[name].apply(lambda val: getId(parent, ref, val, name))

  return newdf

##Entidades

In [ ]:
clasesViejas = ['Equipo/Área','Equipo/Vicepresidencia']
clasesAlteradas = ['area','vicepresidencia']
clasesNuevas = [{'name':'idArea', 'value': True}]

area = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, [])
area.head()

,idArea,area,vicepresidencia
0,d689c2ca797d4b,PLANEACION Y ESTUDIOS TYD (CLS51000895),TRANSMISION Y DISTRIBUCION (CLS80000008)
50,3e1497f1ef324a,TRANSMISION Y DISTRIBUCION (CLS51000577),TRANSMISION Y DISTRIBUCION (CLS80000008)
153,1e6b9de87c7b44,OPERACION DEL MEM (CLS51000848),GENERACION (CLS80000006)
206,5d07bb83108843,PRUEBAS E INGENIERIA TYD (CLS53000571),TRANSMISION Y DISTRIBUCION (CLS80000008)
316,a18a4eb3624d4d,INGENIERIA TYD (CLS51000896),TRANSMISION Y DISTRIBUCION (CLS80000008)


In [ ]:
clasesViejas = ['Categoría de informe','Nombre del trabajo','Equipo/Área']
clasesAlteradas = ['categoriaInforme','nombreTrabajo', 'idArea']
clasesNuevas = [{'name':'idInforme', 'value': True}]
clasesForaneas = [{'parent': area, 'name':'idArea', 'ref':'area'}]

informe = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
informe.head()

,idInforme,categoriaInforme,nombreTrabajo,idArea
0,b61aac60ec514b,OPERATIVE UNIONIZED,PLANEACION Y ESTUDIOS TYD,d689c2ca797d4b
50,93f3a0cca2ed4a,OPERATIVE UNIONIZED,PROYECTOS TYD,3e1497f1ef324a
153,710fa2295bea4b,ADMINISTRATIVE,OPERACION MERCADO MAYORISTA,1e6b9de87c7b44
206,78a58671c0c34a,OPERATIVE UNIONIZED,EQUIPOS DE FLEXIBILIDAD RED DISTRIBUCION,5d07bb83108843
261,91ebb201023147,OPERATIVE,PROYECTOS TYD,3e1497f1ef324a


In [ ]:
clasesViejas = ['País/región']
clasesAlteradas = ['nombre']
clasesNuevas = [{'name':'idPaisRegion', 'value': True},
                {'name': 'tipoPR', 'value':'pais'}]
clasesForaneas = []

pais = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
pais.head()

,idPaisRegion,tipoPR,nombre
0,23f329c24ae94f,e4f0180d556446,Colombia
29140,66b37918e30940,0590a29a159141,Panamá
29671,a04005f3966e4c,f24566bc623042,Costa Rica
121708,ffbc00cf7ac94d,7d1aae1e800841,Honduras


In [ ]:
clasesViejas = ['Estado/Provincia', 'País/región']
clasesAlteradas = ['nombre', 'idPaisRegion']
clasesNuevas = [{'name':'idEstadoProvincia', 'value': True},
                {'name': 'tipoEP', 'value':'estado'}]
clasesForaneas = [{'parent': pais, 'name':'idPaisRegion', 'ref':'nombre'}]

estado = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
estado.head()

,idEstadoProvincia,tipoEP,nombre,idPaisRegion
0,4b3bf027632142,d80d9f31e30c48,TOLIMA,23f329c24ae94f
2745,96ac2c51a6454e,d1b6327a02aa4c,VALLE DEL CAUCA,23f329c24ae94f
3476,688d3d48b34745,694f65f9c39645,ANTIOQUIA,23f329c24ae94f
4643,9992cb7b42da49,4d98926d817143,HUILA,23f329c24ae94f
12698,c667124bfc7946,c6e43e5965ce42,BOLIVAR,23f329c24ae94f


In [ ]:
clasesViejas = ['Ciudad','Estado/Provincia']
clasesAlteradas = ['nombre', 'idEstadoProvincia']
clasesNuevas = [{'name':'idCiudad', 'value': True}]
clasesForaneas = [{'parent': estado, 'name':'idEstadoProvincia', 'ref':'nombre'}]

ciudad = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
ciudad.head()

,idCiudad,nombre,idEstadoProvincia
0,fb9fd780a52c46,AGUA DE DIOS,4b3bf027632142
2745,451b08c2ae164f,AGUA DE DIOS,96ac2c51a6454e
3476,583ea632916249,AGUA DE DIOS,688d3d48b34745
4643,70a57cbe741343,PITALITO,9992cb7b42da49
5201,e566b44ac8b848,JAMBALO,96ac2c51a6454e


In [ ]:
clasesViejas = ['Compañía','Ciudad']
clasesAlteradas = ['nombre','idCiudad']
clasesNuevas = [{'name':'idCompania', 'value': True}]
clasesForaneas = [{'parent': ciudad, 'name':'idCiudad', 'ref':'nombre'}]

compani = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
compani['ubicacionC'] = compani['idCiudad']
compani = compani.drop(['idCiudad'],axis=1)
compani.head()

,idCompania,nombre,ubicacionC
0,efff049730a74e,CELSIA COLOMBIA S.A. E.S.P. (CLS-CO07),fb9fd780a52c46
2580,c4d22330f9304c,ENERBIT S.A.S. E.S.P. (CLS-CO11),fb9fd780a52c46
3642,a22a1838413f41,TERMOELECTRICA EL TESORITO S.A.S E.S.P (CLS-CO10),fb9fd780a52c46
4588,b9e858dc6c1447,FUNDACION CELSIA COLOMBIA (CLS-CO09),fb9fd780a52c46
4643,9bf7387a56d04c,CELSIA COLOMBIA S.A. E.S.P. (CLS-CO07),70a57cbe741343


In [ ]:
clasesViejas = ['Descripción de categoría temática']
clasesAlteradas = ['descripcionCatTem']
clasesNuevas = [{'name':'idCatTem', 'value': True}]
clasesForaneas = []


categoria = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
categoria.head()

,idCatTem,descripcionCatTem
0,0ca94a8d79c845,Academia E4
1,9b8ff1e404b246,Corporativa
2,efa91a81ba9c47,Liderazgo y Gerencia
11,9aa8eba3c28f4c,TI
14,0bf5d92466db43,Diversidad


In [ ]:
clasesViejas = ['Área de conocimiento','Propósito de formación', 'Descripción de categoría temática']
clasesAlteradas = ['nombre','propositoFormacion', 'idCatTem']
clasesNuevas = [{'name':'idAreaCon', 'value': True}]
clasesForaneas = [{'parent': categoria, 'name':'idCatTem', 'ref':'descripcionCatTem'}]


areaCon = restructurar(df_cursos, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
areaCon.head()

,idAreaCon,nombre,propositoFormacion,idCatTem
0,e04868ea8a7a4c,NaN,NaN,9b8ff1e404b246
1,e1f64fb5f3b846,Modelo de procesos,NaN,9b8ff1e404b246
6,6f7d4973cbce48,Proyectos,NaN,9b8ff1e404b246
7,906e359c3afe4d,NaN,NaN,c5d4036ccf7a48
8,3120a57091cb47,NaN,NaN,1f2419b86e604b


In [ ]:
clasesViejas = ['ID de artículo','Área de conocimiento']
clasesAlteradas = ['nombreIdArticulo','idAreaCon']
clasesNuevas = [{'name':'idArticulo', 'value': True}]
clasesForaneas = [{'parent': areaCon, 'name':'idAreaCon', 'ref':'nombre'}]


articulo = restructurar(df_cursos, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
articulo.head()

,idArticulo,nombreIdArticulo,idAreaCon
0,d830a429d5b747,COURSERA_46004,None
1,90530d459a104e,COURSERA_20002,e1f64fb5f3b846
2,53fc0bbad7d742,COURSERA_18009,None
3,e893db59de0f4e,COURSERA_27008,None
4,bea828cc75004c,COURSERA_46003,None


In [ ]:
clasesViejas = ['Título del artículo','Tipo de artículo','Descripción', 'ID de artículo']
clasesAlteradas = ['tituloArticulo','tipoArticulo','descripcion', 'idArticulo']
clasesNuevas = [{'name':'idArticuloEsp', 'value': True}]
clasesForaneas = [{'parent': articulo, 'name':'idArticulo', 'ref':'nombreIdArticulo'}]


articuloEsp = restructurar(df_cursos, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
articuloEsp.head()

,idArticuloEsp,tituloArticulo,tipoArticulo,descripcion,idArticulo
0,8285babca5214a,Strategic Organization Design,VIRTUAL,Strategic Organization Design will introduce n...,d830a429d5b747
1,7ead5784269248,Creativity Toolkit I: Changing Perspectives,VIRTUAL,Thinking and doing the same things faster and ...,90530d459a104e
2,a9eaf547a88045,Sales Strategy,VIRTUAL,Welcome to Course 2 - Sales Strategy - This co...,53fc0bbad7d742
3,f583d057e77d47,Introduction to Scrum Master Training,VIRTUAL,This course is designed to help Scrum beginner...,e893db59de0f4e
4,0c09b669e49649,Foundations of Project Management,VIRTUAL,This course is the first in a series of six to...,bea828cc75004c


In [ ]:
clasesViejas = ['Proveedor']
clasesAlteradas = ['nombre']
clasesNuevas = [{'name':'idProveedor', 'value': True}]
clasesForaneas = []


proveedor = restructurar(df_cursos, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
proveedor.head()

,idProveedor,nombre
0,b67be840714b43,NaN
18,387216c76c7846,Coursera
154,6eb343162ee440,COURSERA
414,806f4f459f9e44,Central Bank of Suriname
415,d765e7178b0445,External


In [ ]:
clasesViejas = ['ID de artículo', 'Título del artículo', 'Tipo de contenido','Contenido del curso', 'Activo', 'Origen', 'Idioma', 'Duración (en horas)', 'Fecha de creación', 'Costo del curso', 'Responsable', 'Experto del curso', 'Proveedor']
clasesAlteradas = ['idArticulo','idArticuloEsp', 'tipoContenido', 'Contenido', 'activo', 'origen', 'idioma', 'duracion', 'fechaCreacion', 'costo', 'responsable', 'experto', 'idProveedor']
clasesNuevas = [{'name': 'Fecha de publicación', 'value': '01/01/2023'}]
clasesForaneas = [{'parent': articulo, 'name':'idArticulo', 'ref':'nombreIdArticulo'},
                  {'parent': proveedor, 'name':'idProveedor', 'ref':'nombre'}]


curso = restructurar(df_cursos, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
curso['idCurso'] = curso['idArticulo']
curso = curso.drop(['idArticulo'],axis=1)
curso.head()

,Fecha de publicación,idArticuloEsp,tipoContenido,Contenido,activo,origen,idioma,duracion,fechaCreacion,costo,responsable,experto,idProveedor,idCurso
0,a4c5386379fc42,Strategic Organization Design,Enlace,NaN,Y,Coursera,Inglés,14,2 dic. 2022,NaN,NaN,NaN,None,d830a429d5b747
1,7892ec3fb70046,Creativity Toolkit I: Changing Perspectives,Enlace,NaN,Y,Coursera,Inglés,NaN,2 jul. 2021,NaN,NaN,NaN,None,90530d459a104e
2,be686163eb2e44,Sales Strategy,Enlace,NaN,Y,Coursera,Inglés,NaN,6 may. 2021,NaN,NaN,NaN,None,53fc0bbad7d742
3,e3e8a60e50894f,Introduction to Scrum Master Training,Registro de horas,NaN,Y,Coursera,Inglés,NaN,5 ene. 2022,NaN,NaN,NaN,None,e893db59de0f4e
4,1be6a38a02e448,Foundations of Project Management,Enlace,NaN,Y,Coursera,Inglés,NaN,2 dic. 2022,NaN,NaN,NaN,None,bea828cc75004c


In [ ]:
curso.to_csv('/content/drive/MyDrive/Bases/curso2.csv', index=False)

In [ ]:
clasesViejas = ['Usuario HR', 'Primer nombre HR','Apellido HR ']
clasesAlteradas = ['usuarioHR', 'primerNombreHR','apellidoHR']
clasesNuevas = []
clasesForaneas = []


recursoHum = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
recursoHum.head()

,usuarioHR,primerNombreHR,apellidoHR
0,28548228,SANDRA,RODRIGUEZ
153,28559710,MARIA,MOREA
206,29817615,SONIA,GARCIA
775,43978362,SANDRA,TAMAYO
981,42692125,LUZ,GONZALEZ


In [ ]:
clasesViejas = ['ID de supervisor', 'Nombre del supervisor','Apellidos del supervisor']
clasesAlteradas = ['idSupervisor', 'nombreSupervisor','apellidoSupervisor']
clasesNuevas = []
clasesForaneas = []


supervisor = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
supervisor.head()

,idSupervisor,nombreSupervisor,apellidoSupervisor
0,16930181.0,MICHAEL,MUÑOZ
50,16792590.0,MAURICIO,LASSO
153,6389015.0,ANDRES,ALAPE
206,94226161.0,CRISTOBAL,GIRALDO
316,94306770.0,JESUS,BETANCOURT


In [ ]:
clasesViejas = ['ID de empleado', 'ID de usuario','Activo', 'Género', 'Primer Nombre', 'Primer Apellido','Grupo de Personal', 'Nombre del trabajo', 'Compañía', 'ID de supervisor', 'Usuario HR', 'Ciudad']
clasesAlteradas = ['idEmpleado', 'idUsuario','activo', 'genero', 'primerNombre', 'primerApellido', 'grupoPersonal', 'idInforme', 'idCompania', 'supervisor', 'recursoHumano', 'idCiudad']
clasesNuevas = []
clasesForaneas = [{'parent': informe, 'name':'idInforme', 'ref':'nombreTrabajo'},
                  {'parent': compani, 'name':'idCompania', 'ref':'nombre'}, 
                  {'parent': ciudad, 'name':'idCiudad', 'ref':'nombre'}]


empleado = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
empleado['ubicacionE'] = empleado['idCiudad']
empleado['compania'] = empleado['idCompania']
empleado['informe'] = empleado['idInforme']
empleado = empleado.drop(['idCiudad','idInforme','idCompania'],axis=1)
empleado.head()

,idEmpleado,idUsuario,activo,genero,primerNombre,primerApellido,grupoPersonal,supervisor,recursoHumano,ubicacionE,compania,informe
0,2000023222,10033390,Y,M,RICARDO,ORTIZ,EMPLOYEE,16930181.0,28548228,fb9fd780a52c46,efff049730a74e,b61aac60ec514b
50,2000023249,1018422710,Y,M,DIEGO,CALDERON,EMPLOYEE,16792590.0,28548228,fb9fd780a52c46,efff049730a74e,93f3a0cca2ed4a
153,2000023331,1026272312,Y,M,JUAN,GIRALDO,EMPLOYEE,6389015.0,28559710,fb9fd780a52c46,efff049730a74e,710fa2295bea4b
206,2000023305,1032376266,Y,M,NIVELSO,GARCIA,EMPLOYEE,94226161.0,29817615,fb9fd780a52c46,efff049730a74e,78a58671c0c34a
261,2000023250,1032447100,Y,M,EFREN,ROJAS,EMPLOYEE,16792590.0,28548228,fb9fd780a52c46,efff049730a74e,93f3a0cca2ed4a


In [ ]:
def getModal(x):
  try:
   return x.replace(" ", "").split('(')[1]
  except:
    return ""

In [ ]:
clasesViejas = ['ID de empleado', 'Calificación', 'Estado de progreso', 'Fecha de finalización', 'Horas totales', 'Usuario de la última actualización', 'Hora de la última actualización', 'ID de artículo']
clasesAlteradas = ['idEmpleado', 'calificacion', 'estado', 'fechaFin', 'totalHoras', 'usuarioUltimaAct', 'horaUltimaAct', 'idArticulo']
clasesNuevas = [{'name':'idCursoEmpleado', 'value': True },
                {'name':'fechaFin', 'value': '01/01/2023' }]
clasesForaneas = [{'parent': articulo, 'name':'idArticulo', 'ref':'nombreIdArticulo'}]


cursoEmpleado = restructurar(df_historico, clasesViejas, clasesAlteradas, clasesNuevas, clasesForaneas)
cursoEmpleado['estado'] = cursoEmpleado['estado'].apply(lambda x: x.replace(" ", "").split('(')[0])
cursoEmpleado['modalidad'] = cursoEmpleado['estado'].apply(lambda x: getModal(x))
cursoEmpleado['idCurso'] = cursoEmpleado['idArticulo']
cursoEmpleado = cursoEmpleado.drop(['idArticulo'],axis=1)
cursoEmpleado.head()

,idCursoEmpleado,fechaFin,idEmpleado,calificacion,estado,totalHoras,usuarioUltimaAct,horaUltimaAct,modalidad,idCurso
0,d96147f35cff43,28/12/2020 22:49:18 América/Nueva York,2000023222,100,Aprobado,2.00,User - 10033390,29 dic. 2020 3:49,,b0a15660af984a
1,3eac27116a724f,25/11/2020 11:50:59 América/Nueva York,2000023222,100,Aprobado,0.33,User - 10033390,25 nov. 2020 16:50,,e3f3968f601c48
2,69b83d846d124d,17/7/2019 13:03:04 América/Nueva York,2000023222,100,Aprobado,0.75,User - 10033390,17 jul. 2019 17:03,,cc498a7d72c046
3,769db86295ac42,9/8/2019 09:24:45 América/Nueva York,2000023222,100,Aprobado,1.00,User - 10033390,9 ago. 2019 13:24,,2bddb8a71d204f
4,5684187d221a4e,25/10/2019 12:51:43 América/Nueva York,2000023222,100,Aprobado,2.00,User - 10033390,25 oct. 2019 16:51,,5b0b7055c4af4e


##Unir

##Exportar

In [ ]:
area.to_csv('/content/drive/MyDrive/Bases/area.csv', index=False)
informe.to_csv('/content/drive/MyDrive/Bases/informe.csv', index=False)
pais.to_csv('/content/drive/MyDrive/Bases/pais.csv', index=False)
estado.to_csv('/content/drive/MyDrive/Bases/estado.csv', index=False)
ciudad.to_csv('/content/drive/MyDrive/Bases/ciudad.csv', index=False)
compani.to_csv('/content/drive/MyDrive/Bases/compani.csv', index=False)
categoria.to_csv('/content/drive/MyDrive/Bases/categoria.csv', index=False)
areaCon.to_csv('/content/drive/MyDrive/Bases/areaCon.csv', index=False)
articulo.to_csv('/content/drive/MyDrive/Bases/articulo.csv', index=False)
articuloEsp.to_csv('/content/drive/MyDrive/Bases/articuloEsp.csv', index=False)
proveedor.to_csv('/content/drive/MyDrive/Bases/proveedor.csv', index=False)
curso.to_csv('/content/drive/MyDrive/Bases/curso.csv', index=False)
recursoHum.to_csv('/content/drive/MyDrive/Bases/recursoHum.csv', index=False)
supervisor.to_csv('/content/drive/MyDrive/Bases/supervisor.csv', index=False)
empleado.to_csv('/content/drive/MyDrive/Bases/empleado.csv', index=False)
cursoEmpleado.to_csv('/content/drive/MyDrive/Bases/cursoEmpleado.csv', index=False)